In [1]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [2]:
import torch
from haystack.utils import clean_wiki_text
from haystack.utils import convert_files_to_docs
from haystack.utils import fetch_archive_from_http
from haystack.utils import print_answers
from haystack.nodes import FARMReader, TransformersReader

h:\models\haystack-test\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# doc_dir = "/content/drive/MyDrive/Asha/text-as-page"
doc_dir = "C:/Users/DELL/Desktop/Asha/text-as-page"
# C:\Users\DELL\Desktop\Asha\text-as-page


# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.

docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

INFO - haystack.utils.preprocessing -  Converting C:\Users\DELL\Desktop\Asha\text-as-page\book-no-4-page-1.txt
INFO - haystack.utils.preprocessing -  Converting C:\Users\DELL\Desktop\Asha\text-as-page\book-no-4-page-10.txt
INFO - haystack.utils.preprocessing -  Converting C:\Users\DELL\Desktop\Asha\text-as-page\book-no-4-page-11.txt
INFO - haystack.utils.preprocessing -  Converting C:\Users\DELL\Desktop\Asha\text-as-page\book-no-4-page-12.txt
INFO - haystack.utils.preprocessing -  Converting C:\Users\DELL\Desktop\Asha\text-as-page\book-no-4-page-13.txt
INFO - haystack.utils.preprocessing -  Converting C:\Users\DELL\Desktop\Asha\text-as-page\book-no-4-page-14.txt
INFO - haystack.utils.preprocessing -  Converting C:\Users\DELL\Desktop\Asha\text-as-page\book-no-4-page-15.txt
INFO - haystack.utils.preprocessing -  Converting C:\Users\DELL\Desktop\Asha\text-as-page\book-no-4-page-16.txt
INFO - haystack.utils.preprocessing -  Converting C:\Users\DELL\Desktop\Asha\text-as-page\book-no-4-page-

In [4]:
import os
from subprocess import Popen, PIPE, STDOUT

# es_server = Popen(
#     ["G:/models/elastic-search/elasticsearch-8.10.4/binelasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)
# )

es_server = Popen(
    ["H:/models/elastic-test/elasticsearch-8.10.4-windows-x86_64/elasticsearch-8.10.4/bin/elasticsearch.bat"], stdout=PIPE, stderr=STDOUT
)


! sleep 30

'sleep' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
from haystack.document_stores import ElasticsearchDocumentStore
# ========================================
# document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document2")
document_store = ElasticsearchDocumentStore(
    host="localhost",  # Use the HTTPS scheme
    username="elastic",
    password="0MNn3loYMbrQrsBz_yDR",
    index="test-book-no-4",
    verify_certs=True,  # Set this to True if you want to verify the SSL certificate
    # ssl_show_warn=False,  # Set this to False if you don't want to show SSL warnings
)

WARNING - elasticsearch -  GET http://localhost:9200/ [status:N/A request:4.116s]
Traceback (most recent call last):
  File "h:\models\haystack-test\.venv\Lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "h:\models\haystack-test\.venv\Lib\site-packages\urllib3\util\connection.py", line 95, in create_connection
    raise err
  File "h:\models\haystack-test\.venv\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "h:\models\haystack-test\.venv\Lib\site-packages\elasticsearch\connection\http_urllib3.py", line 255, in perform_request
    response = self.pool.urlopen(
               ^^^^^^^^^^^^^^^^^^
  File "h:\model

ConnectionError: Initial connection to Elasticsearch failed. Make sure you run an Elasticsearch instance at `[{'host': 'localhost', 'port': 9200}]` and that it has finished the initial ramp up (can take > 30s).

In [ ]:
document_store.write_documents(docs)

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
saved_model_dir = "../haystack-roberta-base-squad2"
reader = FARMReader(model_name_or_path=saved_model_dir, use_gpu=True)


INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0
INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: '../haystack-roberta-base-squad2' (Roberta)
h:\models\haystack-test\.venv\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO - haystack.modeling.model.language_model -  Loaded '../haystack-roberta-base-squad2' (Roberta model) from local file system.
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Migh

In [ ]:
# reader.save(saved_model_dir)

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(
    query="malaria fever treatments", params={"Retriever": {"top_k": 2}, "Reader": {"top_k": 10}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:06<00:00,  6.77s/ Batches]


In [ ]:
prediction

{'query': 'malaria fever treatments',
 'no_ans_gap': -5.577052354812622,
 'answers': [<Answer {'answer': 'Presumptive (inital) treatment for malaria fever \n It should be given to all fever cases if malaria is suspected after taking   blood film wherever possible', 'type': 'extractive', 'score': 0.040137846022844315, 'context': 'Presumptive (inital) treatment for malaria fever \n It should be given to all fever cases if malaria is suspected after taking   blood film wherever possible', 'offsets_in_document': [{'start': 0, 'end': 156}], 'offsets_in_context': [{'start': 0, 'end': 156}], 'document_ids': ['1b40a9306b661d6b2b4552f8568cc64a'], 'meta': {'name': 'book-no-4-page-15.txt'}}>,
  <Answer {'answer': 'headache, bodyache and vomiting. The fever comes down with   profuse sweating and leaves the person with generalized weakness   and discomfort', 'type': 'extractive', 'score': 0.038689810782670975, 'context': 'with headache, bodyache and vomiting. The fever comes down with   profuse swe

In [ ]:
import re

retJson = {}
retJson['res'] = []

for ans in prediction['answers']:
  scr =ans.score
  answer = ans.answer.replace("/square6", "#").replace("\n", "")
  context = re.split("[\n>]", ans.context.replace("/square6", "># ").replace("\n", ""))
  retJson['res'].append({'score': scr, "answer": answer,  "context":context})

In [ ]:
retJson

{'res': [{'score': 0.040137846022844315,
   'answer': 'Presumptive (inital) treatment for malaria fever  It should be given to all fever cases if malaria is suspected after taking   blood film wherever possible',
   'context': ['Presumptive (inital) treatment for malaria fever  It should be given to all fever cases if malaria is suspected after taking   blood film wherever possible']},
  {'score': 0.038689810782670975,
   'answer': 'headache, bodyache and vomiting. The fever comes down with   profuse sweating and leaves the person with generalized weakness   and discomfort',
   'context': ['with headache, bodyache and vomiting. The fever comes down with   profuse sweating and leaves the person with generalized weakness   and discomfort.  ']},
  {'score': 0.003707310650497675,
   'answer': '<p>',
   'context': ['imaquine.  Primaquine should not be given to infants and pregnant women <p',
    'Whenever a case of fever is seen without any other sign/symptom   such as']}]}